# ANOVA 

## Introduction

ANOVA (Analysis of Variance) is a method for generalizing statistical tests to multiple groups. As you'll see, ANOVA  analyses the overall variance of a dataset by partitioning the total sum of square of deviations (from the mean) into sum of squares for each of these groups and sum of squares for error. By comparing statistical test for multiple groups, it can serve as a useful alternative to the $t$-tests you've encountered thus far, when you wish to test multiple factors simultaneously.

## Objectives

You will be able to:
* Use ANOVA for testing multiple pairwise comparisons
* Understand and explain the methodology behind ANOVA tests

### Analysis of Variance:

- The main problem is: Can we determine whether a set of population means are the same or different from samples from these populations.

- Natural Question: Hint: What other test we were using for similar procedure - why can't we use the same (or modified version) here? (Let's count if we have 10 different population how many tests we should do in total?)

- Even if we were able to use above method, what happens to the significance levels?

The main idea: We will compare variances within the group with the total variance to be able to say that whether population means are equal or not.

__Wait!!__ Why use the variance to compare the means?

Let's check this from a scenerio!!

In [ ]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt


In [ ]:
## case1

sample1 = np.random.normal(loc = 6, scale = 0.1, size = 20)

sample2 = np.random.normal(loc = 10, scale = 0.2, size = 20)

sample3 = np.random.normal(loc = 15, scale = 1, size = 20)

## case2

sample4 = np.random.normal(loc = 6, scale = 3, size = 20)
sample5 = np.random.normal(loc = 10, scale = 5, size = 20)
sample6 = np.random.normal(loc = 15, scale = 5, size = 20)

df = pd.DataFrame(data = [sample1, sample2, sample3, sample4, sample5, sample6]).T

df.head()

In [ ]:
df.rename(columns= {0:'s1', 1:'s2', 2:'s3', 3:'s4', 4:'s5', 5:'s6'}, inplace= True)

In [ ]:
import seaborn as sns

In [ ]:
df.boxplot()
plt.tight_layout()

In [ ]:
print('Means of the samples are: ', sample1.mean(), 
      sample2.mean(),
      sample3.mean(), '\n',
      sample4.mean(),
      sample5.mean(),
      sample6.mean())

Note that here even though the variations of $\{s_1, s_2, s_3\}$ equals to the variations of the means in $\{s_4, s_5, s_6\}$ we can see this as a stronger evidence to argue that the population means are different in the first group.

__Summary__ ANOVA: comparison of the variance among the means of the populations to the variance among sample observations within the individual populations.

### Notation

- $t$: total number of populations

- $n_{i}$ number of observations within the sample $i$ drawn from the population $i$.

- $y_{ij}$: the measurement for the $j$th observation from the $i$th sample.

- $\mu_{i}$ is the mean for the $i$th population.

__observations__

$n = \sum\limits_{i} n_{i}$ is the total number of observations

### Hypotheses

-  $H_{0}: \mu_{1} = \mu_{2} = \cdots = \mu_{t}$ 

- $H_{a}$: at least one of the equality is not satisfied

In [ ]:
from scipy.stats import f

In [ ]:
## let's create a data set first


## sample sizes for different groups
n1 =20
n2=20
n3 =20
n4 = 20

## number of groups
t = 4

In [ ]:
np.random.seed(100719)

x1 = np.random.normal(loc = 950, scale = 100, size = n1)

x2 = np.random.normal(loc = 928, scale = np.sqrt(3800), size = n2)

x3 = np.random.normal(loc = 938, scale = np.sqrt(13617), size = n3)

x4 = np.random.normal(loc = 1116, scale = np.sqrt(23000), size = n4)

# df = pd.DataFrame(data = np.stack([x1, x2,x3,x4], axis = 1), columns= ['s1', 's2', 's3', 's4'],
#                   index = np.arange(1, len(x2)+1))

In [ ]:
# df.head()

## Hypothesis

$H_{0}$: $\mu_{1} = \mu_{2} = \mu_{3} = \mu_{4}$

$H_{a}$: not all means are equal.

For more details on how to construct hypotheses testing with ANOVA model please check:

[Penn State - The 7 Step Process of Statistical Hypothesis Testing](https://newonlinecourses.science.psu.edu/stat502/lesson/1/1.2)







In [ ]:
## Let's find these quantities with Python

all_data = np.stack([x1, x2,x3,x4], axis = 1)
overall_mean = all_data.mean()

In [ ]:
## Note that all_data is a (20,4) numpy_ndarray
ss_total = ((all_data - overall_mean)**2).sum()

## Function that calculates between group error
def ss_between_groups(sample_list):
    X_bar = np.stack(sample_list, axis=1).mean()
    terms = [len(sample)*(sample.mean() - X_bar)**2 for sample in sample_list]
    return np.sum(terms)
print(ss_between_groups([x1, x2, x3, x4]))

In [ ]:
## Note that right now we can find the error within the groups
## by simply subtracting ss_between_groups from ss_total

ss_within = ss_total - ss_between_groups([x1, x2, x3, x4])

print(ss_within)

In [ ]:
## Or we can apply the formula:
## A function that calculates within group error
def ss_within_groups(sample_list):
    terms = [(sample - sample.mean())**2 for sample in sample_list]
    return np.sum(terms)

In [ ]:
## Degrees of freedom of numerator
dfn = len([x1, x2, x3, x4]) - 1

## Degree of freedom of denominator
dfd = n1+n2+n3+n4 - len([x1, x2, x3,x4])

In [ ]:
print('Within Group Error', ss_within_groups([x1, x2, x3, x4]))
print('Between Group Error', ss_between_groups([x1, x2, x3, x4]))
print('Mean Squared Error Between Groups', ss_between_groups([x1, x2, x3, x4])/dfn)
print('Mean Squared Error Within Groups', ss_within_groups([x1, x2, x3, x4])/ dfd)

In [ ]:
MSE_between_groups = ss_between_groups([x1, x2, x3, x4])/dfn
MSE_within_groups = ss_within_groups([x1, x2, x3, x4])/ dfd

## F_score comes from the ratio
F_score = MSE_between_groups/ MSE_within_groups
print(F_score)

In [ ]:
## Or we can apply directly scipy.stats's f_oneway method 

import scipy.stats
scipy.stats.f_oneway(x1,x2,x3,x4)

For more details about the notation and formulas please check:

[Penn State](https://newonlinecourses.science.psu.edu/stat502/lesson/2/2.2)